In [5]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/tabular-playground-series-jan-2021/sample_submission.csv
/kaggle/input/tabular-playground-series-jan-2021/train.csv
/kaggle/input/tabular-playground-series-jan-2021/test.csv


In [6]:
import numpy as np 
import pandas as pd

from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import optuna.integration.lightgbm as lgbm
from lightgbm import LGBMRegressor
import seaborn as sns

from tqdm import tqdm

In [7]:
train = pd.read_csv('../input/tabular-playground-series-jan-2021/train.csv')
test = pd.read_csv('../input/tabular-playground-series-jan-2021/test.csv')

In [8]:
train.head()

,id,cont1,cont2,cont3,cont4,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13,cont14,target
0,1,0.670390,0.811300,0.643968,0.291791,0.284117,0.855953,0.890700,0.285542,0.558245,0.779418,0.921832,0.866772,0.878733,0.305411,7.243043
1,3,0.388053,0.621104,0.686102,0.501149,0.643790,0.449805,0.510824,0.580748,0.418335,0.432632,0.439872,0.434971,0.369957,0.369484,8.203331
2,4,0.834950,0.227436,0.301584,0.293408,0.606839,0.829175,0.506143,0.558771,0.587603,0.823312,0.567007,0.677708,0.882938,0.303047,7.776091
3,5,0.820708,0.160155,0.546887,0.726104,0.282444,0.785108,0.752758,0.823267,0.574466,0.580843,0.769594,0.818143,0.914281,0.279528,6.957716
4,8,0.935278,0.421235,0.303801,0.880214,0.665610,0.830131,0.487113,0.604157,0.874658,0.863427,0.983575,0.900464,0.935918,0.435772,7.951046


In [9]:
train.shape

(300000, 16)

In [ ]:
def remove_outlier(df_in, col_name):
    q1 = df_in[col_name].quantile(0.25)
    q3 = df_in[col_name].quantile(0.75)
    iqr = q3-q1 #Interquartile range
    fence_low  = q1-1.5*iqr
    fence_high = q3+1.5*iqr
    df_out = df_in.loc[(df_in[col_name] > fence_low) & (df_in[col_name] < fence_high)]
    return df_out

In [ ]:
df = train.copy()
df1 = remove_outlier(df, 'target')
df2 = remove_outlier(df1, 'cont8')

In [16]:
train = df2.copy()

In [7]:
train.isnull().sum()

id        0
cont1     0
cont2     0
cont3     0
cont4     0
cont5     0
cont6     0
cont7     0
cont8     0
cont9     0
cont10    0
cont11    0
cont12    0
cont13    0
cont14    0
target    0
dtype: int64

In [17]:
train_x = train.drop(['target','id'], 1)
val_y = train['target']

In [18]:
X_train, X_val, y_train, y_val = train_test_split(train_x, val_y) 

In [27]:
dtrain = lgbm.Dataset(X_train, label=y_train)
dval = lgbm.Dataset(X_val, label=y_val)
params = {"objective": "regression","metric": "rmse","verbosity": -1,"boosting_type": "gbdt"}
model = lgbm.train(params, dtrain, valid_sets=[dval], verbose_eval=100, early_stopping_rounds=100, )

[I 2021-01-06 05:34:02,078] A new study created in memory with name: no-name-9fa13aa5-182c-4917-acab-a511573ce174








  0%|          | 0/7 [00:00<?, ?it/s]







feature_fraction, val_score: inf:   0%|          | 0/7 [00:00<?, ?it/s]

Training until validation scores don't improve for 100 rounds
[100]	valid_0's rmse: 0.700362
[200]	valid_0's rmse: 0.698483
[300]	valid_0's rmse: 0.697694
[400]	valid_0's rmse: 0.697429
[500]	valid_0's rmse: 0.697262










feature_fraction, val_score: 0.697193:   0%|          | 0/7 [00:05<?, ?it/s]







feature_fraction, val_score: 0.697193:  14%|#4        | 1/7 [00:05<00:30,  5.12s/it][I 2021-01-06 05:34:07,213] Trial 0 finished with value: 0.6971933218258085 and parameters: {'feature_fraction': 0.7}. Best is trial 0 with value: 0.6971933218258085.








feature_fraction, val_score: 0.697193:  14%|#4        | 1/7 [00:05<00:30,  5.12s/it]

[600]	valid_0's rmse: 0.697319
Early stopping, best iteration is:
[513]	valid_0's rmse: 0.697193
Training until validation scores don't improve for 100 rounds
[100]	valid_0's rmse: 0.70028
[200]	valid_0's rmse: 0.698212
[300]	valid_0's rmse: 0.697716
[400]	valid_0's rmse: 0.697333
[500]	valid_0's rmse: 0.697126
[600]	valid_0's rmse: 0.697105










feature_fraction, val_score: 0.697062:  14%|#4        | 1/7 [00:09<00:30,  5.12s/it]







feature_fraction, val_score: 0.697062:  29%|##8       | 2/7 [00:09<00:25,  5.02s/it][I 2021-01-06 05:34:11,988] Trial 1 finished with value: 0.6970621184022733 and parameters: {'feature_fraction': 0.4}. Best is trial 1 with value: 0.6970621184022733.








feature_fraction, val_score: 0.697062:  29%|##8       | 2/7 [00:09<00:25,  5.02s/it]

...

Training until validation scores don't improve for 100 rounds
[100]	valid_0's rmse: 0.696808
[200]	valid_0's rmse: 0.695751
[300]	valid_0's rmse: 0.695687










min_data_in_leaf, val_score: 0.695384:  60%|######    | 3/5 [00:24<00:12,  6.42s/it]







min_data_in_leaf, val_score: 0.695384:  80%|########  | 4/5 [00:24<00:06,  6.19s/it][I 2021-01-06 05:39:25,553] Trial 63 finished with value: 0.6955866247712502 and parameters: {'min_child_samples': 25}. Best is trial 61 with value: 0.695488302535163.








min_data_in_leaf, val_score: 0.695384:  80%|########  | 4/5 [00:24<00:06,  6.19s/it]

Early stopping, best iteration is:
[242]	valid_0's rmse: 0.695587
Training until validation scores don't improve for 100 rounds
[100]	valid_0's rmse: 0.696514
[200]	valid_0's rmse: 0.695649
[300]	valid_0's rmse: 0.695566










min_data_in_leaf, val_score: 0.695384:  80%|########  | 4/5 [00:30<00:06,  6.19s/it]







min_data_in_leaf, val_score: 0.695384: 100%|##########| 5/5 [00:30<00:00,  6.18s/it]

Early stopping, best iteration is:
[253]	valid_0's rmse: 0.695462


In [28]:
params = model.params
params

{'objective': 'regression',
 'metric': 'rmse',
 'verbosity': -1,
 'boosting_type': 'gbdt',
 'feature_pre_filter': False,
 'lambda_l1': 9.930957696944382,
 'lambda_l2': 0.0015765777300817574,
 'num_leaves': 122,
 'feature_fraction': 0.44800000000000006,
 'bagging_fraction': 1.0,
 'bagging_freq': 0,
 'min_child_samples': 20}

In [29]:
test_ = test.drop(['id'],1)

In [31]:
kf = KFold(n_splits = 5)
oof = np.zeros(len(y_train))
oof_vanilla = np.zeros(len(y_train))
preds = np.zeros(len(test))
params['learning_rate'] = 0.005
params['num_iterations'] = 6000
for train_ind, test_ind in tqdm(kf.split(X_train)):
    Xtrain = X_train.iloc[train_ind]
    Xval = X_train.iloc[test_ind]
    ytrain = y_train.iloc[train_ind]
    yval = y_train.iloc[test_ind]

    model = LGBMRegressor(**params)
    vanilla_model = LGBMRegressor()
    
    model.fit(Xtrain, ytrain, eval_set = ((Xval,yval)), early_stopping_rounds = 100, verbose = 0)
    vanilla_model.fit(Xtrain, ytrain)
    p = model.predict(Xval)
    p_vanilla = vanilla_model.predict(Xval)
    oof[test_ind] = p
    oof_vanilla[test_ind] = p_vanilla
    
    preds += model.predict(test_)/5
    
print(f'mean square error on training data (vanilla model): {np.round(mean_squared_error(y_train, oof_vanilla, squared=False),5)}')    
print(f'mean square error on training data (with optuna tuning): {np.round(mean_squared_error(y_train, oof, squared=False),5)}')









0it [00:00, ?it/s]







1it [02:12, 132.40s/it]







2it [05:33, 153.03s/it]







3it [08:09, 153.85s/it]







4it [10:22, 147.70s/it]







5it [13:19, 159.97s/it]

mean square error on training data (vanilla model): 0.70098
mean square error on training data (with optuna tuning): 0.69506


In [32]:
submission = test[['id']]
submission['target'] = preds
submission.to_csv('submission3.csv', index = False)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
